In [0]:
catalog_name = "dmc_tallerfinaldatabricks_mariovento"

schema_bronze = "bronze"
schema_silver = "silver"

In [0]:
from pyspark.sql.functions import col, when, concat, lit, initcap, trim

In [0]:
silver_tiendas = (
  spark.table(f"{catalog_name}.{schema_bronze}.tiendas_raw")  # Lee la tabla de tiendas en bruto
  .withColumn("id_tienda", col("id_tienda").cast("bigint"))  # Convierte el id de tienda a tipo bigint
  .withColumn("nombre",
        # Si el nombre es nulo, lo reemplaza por "Tienda <id_tienda>"
        when(col("nombre").isNull(), concat(lit("Tienda"), lit(" "), col("id_tienda").cast("string")))
        # Si el nombre contiene "tienda" (sin importar mayúsculas/minúsculas), lo capitaliza y recorta espacios
        .when(col("nombre").rlike("(?i).*tienda.*"), initcap(trim(col("nombre"))))
        # En otros casos, capitaliza y recorta el nombre y le agrega el id de tienda
        .otherwise(concat(initcap(trim(col("nombre"))), lit(" "), col("id_tienda").cast("string")))
  )
  # Capitaliza y recorta los campos ciudad y región, y los convierte a string
  .withColumn("ciudad", initcap(trim(col("ciudad"))).cast("string"))
  .withColumn("region", initcap(trim(col("region"))).cast("string"))
  # Rellena valores nulos en ciudad y región con "Desconocido"
  .fillna("Desconocido", subset=["ciudad", "region"])
  # Elimina duplicados por id_tienda
  .dropDuplicates(["id_tienda"])
)

In [0]:
display(silver_tiendas.groupBy("ciudad").count())

In [0]:
display(silver_tiendas.groupBy("region").count())

In [0]:
display(silver_tiendas)

In [0]:
silver_tiendas.write.mode("overwrite").saveAsTable(f"{catalog_name}.{schema_silver}.tiendas")